<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/TerminologyExtractionV01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install morphological analysis tools


In [ ]:
%%bash
# installing TreeTagger (en, de, ka)
mkdir treetagger
cd treetagger
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/english.par.gz
sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper
# changing options: no-unknown, sgml, lemma
mv /content/treetagger/cmd/tree-tagger-english /content/tree-tagger-english0
awk '{ if (NR == 9) print "OPTIONS=\"-token -lemma -sgml -no-unknown\""; else print $0}' /content/tree-tagger-english0 > /content/treetagger/cmd/tree-tagger-english
chmod a+x ./treetagger/cmd/tree-tagger-english
# downloading German and Georgian 
wget https://heibox.uni-heidelberg.de/f/ec8226edebb64a359407/?dl=1
mv index.html?dl=1 /content/treetagger/lib/german-utf8.par
wget https://heibox.uni-heidelberg.de/f/9183090d2bdb41e09055/?dl=1
mv index.html?dl=1 /content/treetagger/lib/georgian.par
wget https://heibox.uni-heidelberg.de/f/9cafab0509d64ed1ac4b/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-georgian2
# German2 = -no-unknown 
# note: tree-tagger-german will not work, as parameter files have not been downloaded, only use tree-tagger-german2 with utf8 encoding
wget https://heibox.uni-heidelberg.de/f/acb9b8a2fa4f40e08f8a/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-german2
chmod a+x /content/treetagger/cmd/tree-tagger-georgian2
chmod a+x /content/treetagger/cmd/tree-tagger-german2


In [ ]:
!pip install spacy-stanza

import stanza
import spacy_stanza

# optional
# Download the stanza model if necessary
stanza.download("en")

# Initialize the pipeline
nlp = spacy_stanza.load_pipeline("en")

doc = nlp("Barack Obama was born in Hawaii. He was elected president in 2008.")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.ent_type_)
print(doc.ents)


stanza.download("hy")

nlp_hy = spacy_stanza.load_pipeline("hy")

doc = nlp_hy("ՄԱՐԴՈՒ ԻՐԱՎՈՒՆՔՆԵՐԻ ՀԱՄԸՆԴՀԱՆՈՒՐ ՀՌՉԱԿԱԳԻՐ. ՆԵՐԱԾԱԿԱՆ. Քանզի մարդկային ընտանիքի բոլոր անդամներին ներհատուկ արժանապատվությունըև հավասար ու անօտարելի իրավունքները աշխարհի ազատության, արդարության ու խաղաղության հիմքն են.")


### optional
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.ent_type_)


def parseFile(iFileName, oFileName, nlp_model = nlp_hy):
    with open(iFileName, 'r', encoding='utf-8') as infile, open(oFileName, 'w') as outfile:
        # read sample.txt an and write its content into sample2.txt
        outfile.write("{token.text}\t{token.pos_}\t{token.lemma_}\n")
        c = 0
        for line in infile:
            c+=1
            if c%10 == 0: print(str(c))
            line = line.strip()
            doc = nlp_model(line)
            # outfile.write(line + '\n')
            for token in doc:
                LAncestors = list(token.ancestors)
                # print(str(LAncestors))
                try:
                    SLAncestors = str(list(token.ancestors))
                    parent = LAncestors[0]
                    parentLem = parent.lemma_
                except:
                    parentLem = "NONE"
                outfile.write(f"{token.text}\t{token.pos_}\t{token.lemma_}\n")
 
    return

!wget https://heibox.uni-heidelberg.de/f/a847a12bffd4491f9070/?dl=1
!mv index.html?dl=1 TED2020-dehy-hy-aa


In [ ]:
parseFile('/content/TED2020-dehy-hy-aa', '/content/TED2020-dehy-hy-aa--lemmatization-v01.txt', nlp_hy)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150


/usr/local/lib/python3.8/dist-packages/spacy/language.py:1014: UserWarning: Due to multiword token expansion or an alignment issue, the original text has been replaced by space-separated expanded tokens.
  doc = self._ensure_doc(text)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:1014: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['Դուք', 'երբևէ', 'ծանրաբեռնված', 'զգացել', '՞', 'եք', 'մի', 'որևէ', 'բարդ', 'խնդրի', 'հանդիպելիս', ':']
Entities: []
  doc = self._ensure_doc(text)


160
170


/usr/local/lib/python3.8/dist-packages/spacy/language.py:1014: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['Ակնհայտ', 'է', ',', 'որ', 'բարդ', 'առաջադրանք', 'է', ',', 'բայց', 'արդյոք', '՞', 'այն', 'խճճված', 'է', ':']
Entities: []
  doc = self._ensure_doc(text)


180
190
200


...

870
880
890
900
910
920
930
940
950


/usr/local/lib/python3.8/dist-packages/spacy/language.py:1014: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['Քիչ', 'էր', 'մնում', 'ասեի', '(', 'չնայած', 'չասեցի', ')', '..', 'ուզում', 'էի', 'ասել', '.', '«', 'գիտես', '՞', 'ինչ', ':', 'Եթե', 'գոնե', 'մի', 'հինգ', 'ժամ', 'քնեիր', ',', 'այս', 'ճաշը', 'գուցե', 'շատ', 'ավելի', 'հետաքրքիր', 'լիներ', '»', ':']
Entities: []
  doc = self._ensure_doc(text)
/usr/local/lib/python3.8/dist-packages/spacy/language.py:1014: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['Հատկապես', 'այստեղ', ',', 'Վաշինգտոնում', ',', 'եթե', 'որևէ', 'մեկի', 'հրավիրել', 'նախաճաշի', ',', 'և', 'ասել', '.', '«', 'Հանդիպենք', 'ժամը', '<UNK><UNK>ին', '՞', '»', ':']
Entities: []
  doc = self._ensure_doc(text)


960
970
980


/usr/local/lib/python3.8/dist-packages/spacy/language.py:1014: UserWarning: Can't set named entities because of multi-word token expansion or because the character offsets don't map to valid tokens produced by the Stanza tokenizer:
Words: ['Մոտավորապես', '1852', 'թ', '.', 'նրանք', 'մտածում', 'էին', '.', '&lt;&lt', '<UNK>Գուցեե', '՞', 'ես', 'աշխարհի', 'ամենահիմար', 'մարդն', 'եմ', ',', 'որ', 'չեմ', 'շտապում', 'Կալիֆորնիա', ':', '&gt;&gt', ';Եվ', 'սկսում', 'են', 'մտածել', ',', 'որ', 'իրոք', 'այդպիսին', 'են', ':']
Entities: []
  doc = self._ensure_doc(text)


990


In [ ]:
%%bash
wget https://heibox.uni-heidelberg.de/f/95a3875771c040db959a/?dl=1
mv index.html?dl=1 humanrights02.txt

wget https://heibox.uni-heidelberg.de/f/a79b829e15c24dbd9e77/?dl=1
mv index.html?dl=1 covid3m-en.txt

wget https://heibox.uni-heidelberg.de/f/e3c1edbcec9649f5b4c4/?dl=1
mv index.html?dl=1 TED2020-enka-en.txt

!wget https://heibox.uni-heidelberg.de/f/cdf240db84ca4718b718/?dl=1
!mv index.html?dl=1 go1984.txt


In [ ]:

!./treetagger/cmd/tree-tagger-english humanrights02.txt >humanrights02_vert.txt

!./treetagger/cmd/tree-tagger-english covid3m-en.txt >covid3m-en_vert.txt

!./treetagger/cmd/tree-tagger-english TED2020-enka-en.txt >TED2020-enka-en_vert.txt

!./treetagger/cmd/tree-tagger-english go1984.txt >go1984_vert.txt

In [ ]:
!head --lines=20 humanrights02_vert.txt

In [ ]:
!head --lines=20 TED2020-enka-en_vert.txt

In [ ]:
!head --lines=20 covid3m-en_vert.txt

## Terminology extraction scripts


In [7]:
import os, re, sys, time
def printDict(DictionaryFrq, FileOut, FileOut1):
    for Word, Frq in sorted( DictionaryFrq.items() , key=lambda x: x[1], reverse=True):
        if re.search(' ', Word):
            FileOut.write(Word + '\t' + str(Frq) + '\n')
        else:
            FileOut1.write(Word + '\t' + str(Frq) + '\n')

    FileOut.flush()
    FileOut1.flush()
        


def openFiles(SFIn, SFOut, SFOut1):
    FInStream = open(SFIn, 'r')
    FOutStream = open(SFOut, 'w')
    FOutStream1 = open(SFOut1, 'w')
    return FInStream, FOutStream, FOutStream1


class clProcCorpus(object):
    ''' we will read a text file and return a dictionary
    this will be done on the line by line basis
    The dictionary can be sorted later...
    '''
    # this is a class for processing a corpus

    def __init__(self, FileIN):
        self.DictFrq = {}
        self.processCorpus(FileIN)

    def processCorpus(self, FileIN):
        LTerm = []
        k = 0
        for Line in FileIN:
            k+=1
            if k%500000 == 0: print(str(k))
            Line = Line.strip()
            LLine = re.split('\t', Line)
            try:
                Word = LLine[0]
                PoS = LLine[1]
                Lemma = LLine[2]
            except:
                Word = ""
                PoS = ""
                Lemma = ""
            
      #Select the Tags for your langauge
            #if re.match('N.*', PoS) or re.match('A.*', PoS): #Arm
            #if re.match('N.*', PoS) or re.match('ADJ.*', PoS): #DE
            if re.match('N.*', PoS) or re.match('J.*', PoS): #EN

      #Terms as Words or Lemmas
                LTerm.append(Word)
                # LTerm.append(Lemma)
            else:
                STerm = ' '.join(LTerm)
                LTerm = []

                try:
                    self.DictFrq[STerm] += 1
                except:
                    self.DictFrq[STerm] = 1        

        return

In [8]:

FIn, FOut, FOut1 = openFiles('humanrights02_vert.txt', 'humanrights02_terms.txt', 'humanrights02_terms1.txt')


In [9]:

FIn, FOut, FOut1 = openFiles('covid3m-en_vert.txt', 'covid3m-en_terms.txt', 'covid3m-en_terms1.txt')

In [ ]:

FIn, FOut, FOut1 = openFiles('TED2020-enka-en_vert.txt', 'TED2020-enka-en_terms.txt', 'TED2020-enka-en_terms1.txt')

In [ ]:
OCorpus = clProcCorpus(FIn)
printDict(OCorpus.DictFrq, FOut, FOut1)
